In [23]:
%pip install transformers datasets peft protobuf==3.20

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [24]:
from datasets import load_dataset

# List of dataset splits
splits = ["cos_e_v1.11_aligned_with_common_sense", "cos_e_v1.11_description_question_option_id", 
          "cos_e_v1.11_description_question_option_text", "cos_e_v1.11_explain_why_human", 
          "cos_e_v1.11_generate_explanation_given_text", "cos_e_v1.11_i_think", 
          "cos_e_v1.11_question_description_option_id", "cos_e_v1.11_question_description_option_text", 
          "cos_e_v1.11_question_option_description_id", "cos_e_v1.11_question_option_description_text", 
          "cos_e_v1.11_rationale"]


In [25]:
from peft import PromptTuningConfig, TaskType, get_peft_model
#load the model and tokenizer
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

prompt_tuning_config = PromptTuningConfig(
    task_type=TaskType.SEQ_CLS,
    num_virtual_tokens=10
)

peft_model = get_peft_model(model, prompt_tuning_config)

In [36]:
# Load and print the first few examples from each split
split_data = {}
for split in splits:
    split_data[split] = load_dataset("bigscience/P3", split)
    print(f"First few examples from {split}:")
    print(split_data[split].keys())
    print(split_data[split]['train'][:5])
    print(split_data[split]['validation'][:5])

data = split_data['cos_e_v1.11_aligned_with_common_sense']


First few examples from cos_e_v1.11_aligned_with_common_sense:
dict_keys(['train', 'validation'])
{'inputs': [[947, 31, 7, 3, 9, 822, 11, 3, 9, 360, 487, 4269, 10, 1593, 10, 96, 7238, 33, 335, 16981, 30, 46, 8947, 2195, 5, 5245, 1590, 326, 5, 852, 132, 33, 3, 4, 16981, 535, 363, 19, 48, 46, 677, 13, 58, 29403, 71, 10, 2447, 6, 7089, 484, 6, 2004, 1530, 6, 7270, 682, 6, 18076, 1615, 19, 96, 3357, 107, 682, 121, 46, 1525, 7901, 15, 26, 28, 936, 1017, 1254, 58], [947, 31, 7, 3, 9, 822, 11, 3, 9, 360, 487, 4269, 10, 1593, 10, 71, 1079, 19, 3, 9, 8524, 51, 5, 8718, 114, 8, 26524, 6, 3, 88, 1342, 1084, 48, 1843, 13, 5127, 3620, 5, 2840, 405, 3, 88, 619, 58, 29403, 71, 10, 2601, 14089, 6, 2608, 6, 2412, 2478, 6, 4716, 6, 4716, 1615, 19, 96, 9818, 121, 46, 1525, 7901, 15, 26, 28, 936, 1017, 1254, 58], [947, 31, 7, 3, 9, 822, 11, 3, 9, 360, 487, 4269, 10, 1593, 10, 71, 1282, 568, 1747, 385, 701, 30, 271, 5057, 6, 6922, 406, 7140, 5167, 42, 271, 125, 58, 29403, 71, 10, 1287, 6, 23868, 6, 3331, 6

In [38]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    logging_dir="./logs",
)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=data['train'],
    eval_dataset=data['validation'],
)

trainer.train()

trainer.evaluate()


IndexError: Invalid key: 9535 is out of bounds for size 0